# Open devkit

use Python 3.6.9

In [ ]:
import sinabs.backend.dynapcnn.io as sio
devkit = sio.open_device("speck2fdevkit:0")

# Build samna graph
## Build a graph in samna to show dvs
import samna

samna_graph = samna.graph.EventFilterGraph()

_, _, streamer = samna_graph.sequential([
    devkit.get_model_source_node(),  # Specify the source of events to this graph as the devkit
    "Speck2fDvsToVizConverter",   # Convert the events to visualizer events
    "VizEventStreamer",  # Stream events to a visualizer via a streamer node
])

"""Creat the graph branch for reading the DVS event directly from the sensor"""
dvs_sensor_buffer_node = samna.BasicSinkNode_speck2f_event_output_event()
_, filter_dvs, _ = samna_graph.sequential(
    [
        devkit.get_model_source_node(),
        "Speck2fOutputEventTypeFilter",
         dvs_sensor_buffer_node,
    ]
)
# set desired type of the filter node
filter_dvs.set_desired_type("speck2f::event::DvsEvent")

## Launch visualizer window
#### Specify the tcp port of the visualizer
visualizer_port = "tcp://0.0.0.0:40000"

### Launch visualizer
gui_process = sio.launch_visualizer(receiver_endpoint=visualizer_port, disjoint_process=True)

## Setpu the visualizer
### Visualizer configuration branch of the graph.
visualizer_config, _ = samna_graph.sequential([
    samna.BasicSourceNode_ui_event(),  # For generating UI commands
    streamer
])

# Connect to the visualizer
streamer.set_streamer_destination(visualizer_port)
if streamer.wait_for_receiver_count() == 0:
    raise Exception(f'Connecting to visualizer on {visualizer_port} fails.')

# Specify which plot is to be shown in the visualizer
plot1 = samna.ui.ActivityPlotConfiguration(image_width=128, image_height=128, title="DVS Layer", layout=[0, 0, 1, 1])
visualizer_config.write([
    samna.ui.VisualizerConfiguration(plots=[plot1])
])

# Try get_raw_result() method
event_graph = samna.graph.EventFilterGraph()
_, rater = event_graph.sequential([
    devkit.get_model_source_node(),  # Specify the source of events to this graph as the devkit
    "Speck2fBackgroundNoiseRater", 
])

# Try get_events() method
buf = samna.graph.sink_from(devkit.get_model().get_source_node())

# Launch the devkit and start visualization

In [ ]:
# Start DVS camera
samna_graph.start()
devkit_config = samna.speck2f.configuration.SpeckConfiguration()
# enable monitoring the inputs from the DVS sensor 
devkit_config.dvs_layer.raw_monitor_enable = True
# Apply this configuration
devkit.get_model().apply_configuration(devkit_config)

while True:
    dvs_events = dvs_sensor_buffer_node.get_events()
    if len(dvs_events) != 0:
        print(dvs_events)

#  Stop the devkit and visualization

In [ ]:
# Stop the graph
samna_graph.stop()

# If we used a sub-process to launch the visualizer, use that to terminate the visualizer. 
if gui_process:
    gui_process.terminate()
    gui_process.join()